# Table of Contents
* [Introduction](#introduction)
    - [About the Project](#project)
    - [About the Data](#data)
* [Getting Started](#getting-started)
    - [Import Python Libraries](#import-libraries)
    - [Import Data](#import-data)
* [Exploratory Data Analysis (EDA)](#eda)
    - [EDA Technical Insights](#eda-tec)
    - [EDA Managerial Insights](#eda-man)
* [RFM Analysis](#rfm)
    - [RFM Technical Insights](#rfm-tec)
    - [RFM Managerial Insights](#rfm-man)
* [Resources & References](#reference)

<a id="introduction"></a>
# Introduction
### About the Project
E-Commerce Data Analysis & Customer Segmentation (EDA & RFM)

In this **Jupyter notebook**, we're going to use **Python** and perform some **exploratory data analysis (EDA)** along with some machine learning techniques. I've tried to make the notebook beginner friendly, so if you are new to the whole world of data analysis, data science and machine learning, you can enjoy it too.

### About the Data
I've used a [Kaggle dataset](https://www.kaggle.com/datasets/carrie1/ecommerce-data?datasetId=1985) for this notebook. Originially, the dataset is provided by the The [UCI Machine Learning Repository](https://archive-beta.ics.uci.edu/ml/datasets/online+retail) and contains actual e-commerce transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based online retailer. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

**What do the instances (aka columns/variables/features) that comprise the dataset represent?**

- **InvoiceNo:** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
- **StockCode:** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- **Description:** Product (item) name. Nominal.
- **Quantity:** The quantities of each product (item) per transaction. Numeric.	
- **InvoiceDate:** Invice Date and time. Numeric, the day and time when each transaction was generated.
- **UnitPrice:** Unit price. Numeric, Product price per unit in sterling.
- **CustomerID:** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- **Country:** Country name. Nominal, the name of the country where each customer resides. 

<a id="getting-started"></a>
# Getting Started
<a id="import-libraries"></a>
### Import Python Libraries
Here are the python libraries I'd like to import:
- **NumPy** for linear algebra
- **Pandas** for tabular data manipulation & processing, CSV file I/O
- **DataPrep** for exploratory data analysis and data cleaning (built on top of Pandas, Matplotlib, Jupyter Widgets, Bokeh, Flask, Dask, etc.)
- **Matplotlib** for data visualization (scatterplots, bar charts, histograms, etc.)
- **Seaborn** for data visualization and statistical plotting (built on top of Matplotlib)
- **Plotly** for interactive data visualization
- **Scikit-learn** for data modeling and machine learning

In [2]:
!pip install dataprep
!pip install -qq plotly_express

In [3]:
import numpy as np
import pandas as pd
import datetime as dt
from dataprep.eda import plot, plot_missing, plot_correlation, create_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

<a id="import-data"></a>
### Import Data
As mentioned in the introduction, data is pooled from a Kaggle e-commerce dataset. You can learn more on how to use Kaggle Datasets <a href="https://www.kaggle.com/docs/notebooks#datasets">here</a>.

In [4]:
df = pd.read_csv("../input/ecommerce-data/data.csv", encoding = 'ISO-8859-1')

<a id="eda"></a>
# Exploratory Data Analysis (EDA)

Let's explore:
- Column names and their associated data types
- Total number of columns & rows
- Duplicate values
- Missing (null) values
- Negative values
- Data consistency
- Basic statistics for quantitative variables (Count, Mean, Min, Median, Max)


In [5]:
# Changing the invoice dates data type to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
plot(df)

<a id="eda-tec"></a>
## EDA Technical Insights

**Columns & Rows**
- As expected, there are 8 Columns (features/variables).
- There are 541909 Rows.
 
**Duplicate Values**
- There are 5268 (1%) duplicate rows.
    - **Must be removed before further analysis.**

**Missing Values**
- There are 136534 (3.1%) missing cells.
- Only 2 columns have missing values: **CustomerID** (135080 or 24.93%) and **Description** (1454 or 0.27%)
    - **Missing "CustomerID" Must be excluded before further analysis.**

**Negative Values**
- **Quantity** has 10624 (1.96%) negative values
    - Some of these are associated with **cancelled orders**. Must be indentified and removed before further analysis.
    - Anything beyond cancelled orders **must be excluded before further analysis.**

**Skew** 
- Both **Quantity** and **UnitPrice** are skewed to the right (towards higher amounts).

**High Cardinality** 
- **InvoiceNo** has a high cardinality: 25900 distinct values
- **StockCode** has a high cardinality: 4070 distinct values
- **Description** has a high cardinality: 4223 distinct values
- **InvoiceDate** has a high cardinality: 23260 distinct values


In [6]:
# change columns type - String to Int type 
#df['CustomerID'] = df['CustomerID'].astype('int64')

# Excluding missing values
df.dropna(subset = ['CustomerID'], axis = 0, inplace = True)
df = df[df.isin(["NaN","missing","?","??"]).any(axis=1) == False]

# Excluding Duplicates
df = df.drop_duplicates()

# Canceled Orders can be further explored
# df_cancelled = df[df['InvoiceNo'].str.contains('C', regex=True)]['InvoiceNo'].unique()

# Ensuring "UnitPrice" & "Quantity" only have positive values
df = df[(df["UnitPrice"] > 0) & (df["Quantity"] > 0)]

# Ensuring Data Consistency for the "Description" and excluding irrelevant values
df["Description"] = df["Description"].str.lower().str.strip()
df = df[df["Description"].isin(["amazon fee", "samples", "postage", "packing charge","manual","discount","adjust bad debt","bank charges","cruk commission","next day carriage"]) == False]

# Optional: Country names can be cleaned if it's confusing to the audience
# e.g.: RSA = South Africa / EIRE = Ireland
# df.drop(["Unspecified","European Community"], axis = 0, inplace = True)

# Total Purcahse is an important quantittive variable (TotalPurchase = Price * Quantity)
df['TotalPurchase'] = df['Quantity'] * df['UnitPrice']

In [ ]:
# Total Purchase by each Country can be further explored
#df = df.groupby("Country").agg({'TotalPurchase': lambda x: x.sum()})

# Observing the Order Size per Invoice
df_orders = df.groupby(by=['CustomerID', 'InvoiceNo', 'TotalPurchase'], as_index=False)['Description'].count().sort_values(by=['Description'], ascending = False)
df_orders = df_orders.rename(columns = {'Description':'OrderSize'})
df_orders.sort_values(by=['OrderSize'], ascending = False)

In [12]:
df.head()

<a id="eda-man"></a>
## EDA Managerial Insights

In [ ]:
plot(df)

In [ ]:
# Total Purchase by each Country can be further explored
#df = df.groupby("Country").agg({'TotalPurchase': lambda x: x.sum()})

# Observing the Order Size per Invoice
df_orders = df.groupby(by=['CustomerID', 'InvoiceNo', 'TotalPurchase'], as_index=False)['Description'].count().sort_values(by=['Description'], ascending = False)
df_orders = df_orders.rename(columns = {'Description':'OrderSize'})
df_orders.sort_values(by=['OrderSize'], ascending = False)

- **Above figures showcase:**
    - Majority of purcahses (89.17%) made from the the UK. 
    - Total purchase figures made by different countries
    - Germany, France, Ireland (EIRE), Spain, Netherlands follow UK in terms of the number of purchases.
    - Top 10 StockIDs
    - Top 10 products
    - Busiest month of the year (November)


- **Note to DE (Data Engineer)/DBA (Database Administrator)/ Upstream**
    - "StockCode" and "Description": Each "StockCode" must have a unique "Description". Any further comments must be documented under a sepaate column (e.g. "Comments").
    - Cancelled Orders should have a separate note on a dedicated column (e.g. "Comments") besides adding a C to the "InvoiceID" or messing with the "Description"

<a id="rfm"></a>
# RFM Analysis

RFM (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.

- Recency (R): How recently purchase was made.
- Frequency (F): How frequently purchase is made.
- Monetary Value (M): How much is spent.


**Business Implications**

Based on the assigned RFM score, customers are divided into 11 segments:

- **Champion:** Bought recently, order often and spend the most.
- **Loyal:** Orders regularly. Responsive to promotions.
- **Potential Loyalist:** Recent customers, and spent a good amount.
- **New Customers:** Bought most recently.
- **Promising:** Potential loyalist a few months ago. Spends frequently and a good amount. But the last purchase was several weeks ago.
- **Core:** Standard customers with not too long-ago purchase.
- **Needs attention:** Core customers whose last purchase happened more than one month ago.
- **Can’t lose them but losing:** Made the largest orders, and often. But haven’t returned for a long time.
- **At Risk:** Similar to “Can’t lose them but losing” but with smaller monetary and frequency value.
- **Losing but engaged:** Made their last purchase a long time ago but in the last 4 weeks either visited the site or opened an email.
- **Lost:** Made last purchase long time ago and didn’t engage at all in the last 4 weeks.

In [ ]:
reference_date = dt.datetime(2011, 12, 12)

rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda InvoiceDate: (reference_date - InvoiceDate.max()).days,
                                     'InvoiceNo': lambda InvoiceNo: InvoiceNo.nunique(),
                                     'TotalPurchase': lambda TotalPurchase: TotalPurchase.sum()})

rfm.columns = ['recency', 'frequency', 'monetary']

# Setting Recency Score: most recent/latest purchase is assigned 5 (high value)
rfm["RecencyScore"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

# Setting Frequency Score: most frequent is assigned 5 (high value)
rfm["FrequencyScore"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

# Setting Monetary Score: highest monetary purchase is assigned 5 (high value)
rfm["MonetaryScore"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

#we'll not include monetory_score.
rfm["RFM_Score"] = (rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str))

seg_map = {
    r'[4-5][4-5][4-5]': 'champion',
    r'[3-5][3-5][1-3]': 'loyal',
    r'[4-5][1-3][3-5]': 'potential_loyalist',
    r'[4-5][1-2][1-5]': 'new_customers',
    r'[2-3][1-3][3-5]': 'promising',
    r'[2-3][2-3][2-3]': 'core',
    r'[1-2][2-3][1-3]': 'need_attention',
    r'[1-3][3-5][4-5]': 'cant_lose',
    r'[1-2][2-5][1-3]': 'at_Risk',
    r'[2-3][1-2][1-5]': 'losing_Engaged',
    r'1[1-2][1-5]': 'lost'
}
rfm['segment'] = rfm['RFM_Score'].replace(seg_map, regex=True)
rfm.reset_index(inplace=True)

<a id="rfm-tec"></a>
## RFM Technical Insights

Each of the RFM components is assigned a score between 1 and 5 (5 being the best score). 

**The Recency (R), Frequency (F), Monetary (M) Breakdown:**
- Champion [R(4 – 5), F(4 – 5), M(4 - 5)]
- Loyal  [R(3 – 5), F(3 – 5), M(1 - 3)]
- Potential Loyalist [R(4 – 5), F(1 – 3), M(3 - 5)]
- New Customers R [(4 – 5), F(1 – 2), M(1 - 5)]
- Promising [R(2 – 3), F(1 – 3), M (3 - 5)]
- Core [R(2-3), F(2-3), M(2-3)]
- Need Attention R [(1 – 2), F(2 – 3), M(1 - 3)]
- Can’t Lose but Losing [R(1 – 3), F(3 – 5), M(4 - 5)]
- At Risk [R(1 – 2), F(2 – 5), M(1 - 3)]
- Losing but Engaged [R(2 – 3), F(1 - 2), M(1 - 5)]
- Lost [R(1), F(1 – 2), M(1 - 5)]

In [ ]:
segments_count = rfm.groupby("segment").agg({"CustomerID": "count"})
segments_count.reset_index(inplace=True)
segments_count.columns = ['segment', 'count']
segments_count.sort_values(by=["count"], ascending = False)

<a id="rfm-man"></a>
## RFM Managerial Insights

In [ ]:
px.treemap(segments_count, path=[px.Constant("segment"),'segment'], values='count',
                 color='count', hover_data=['count'], 
                 color_continuous_scale='RdBu',
                 color_continuous_midpoint=np.average(segments_count['count'], weights=segments_count['count']))

**How to treat each RFM segment**

- **Champion:** Reward them. Can be early adopters of new products. Will promote your brand. Most likely to send referrals.
- **Loyal:** Upsell higher value products. Ask for reviews.
- **Potential Loyalist:** Offer membership / loyalty program. Keep them engaged. Offer personalised recommendations.
- **New Customers:** Provide on-boarding support, give them early access, start building relationship.
- **Promising:** Offer coupons. Bring them back to the platform and keep them engaged. Offer personalised recommendations.
- **Core:** Make limited time offers.
- **Needs attention:** Make limited time offers. Offer personalised recommendations.
- **Can’t lose them but losing:** Win them back via renewals or newer products, don’t lose them to competition. Talk to them if necessary. Spend time on highest possible personalisation.
- **At Risk:** Provide helpful resources on the site. Send personalised emails.
- **Losing but engaged:** Make subject lines of emails very personalised. Revive their interest by a specific discount on a specific product.
- **Lost:** Revive interest with reach out campaign. Ignore otherwise.


<a id="reference"></a>
# Resources & References

**Python Libraries' User Guides & Tutorials**
- <a href="https://numpy.org/doc/stable/">Numpy Getting Started and User Guide</a>
- <a href="https://pandas.pydata.org/docs/">Pandas Getting Started and User Guide</a>
- <a href="https://matplotlib.org/stable/users/index.html">Matplotlib User Guide</a>
- <a href ="https://seaborn.pydata.org/tutorial.html">Seaborn: Statistical Data Visualization. User Guide and Tutorial</a> 
- <a href="https://plotly.com/python/getting-started/">Plotly Getting Started in Python</a>
- <a href="https://docs.dataprep.ai/user_guide/user_guide.html">DataPrep User Guide</a>
- <a href ="https://scikit-learn.org/stable/user_guide.html">Scikit-learn: Machine Learning in Python. User Guide</a>
- <a href="https://docs.scipy.org/doc/scipy/">SciPy Getting Started and User Guide</a>


**Kaggle Courses**
- <a href ="https://www.kaggle.com/learn/python">Kaggle Courses: Python</a> (beginner friendly)
- <a href ="https://www.kaggle.com/learn/pandas">Kaggle Courses: Pandas</a> (beginner friendly)
- <a href ="https://www.kaggle.com/learn/data-cleaning">Kaggle Courses: Data Cleaning</a> (beginner friendly)
- <a href ="https://www.kaggle.com/learn/data-visualization">Kaggle Courses: Data Visualization</a> (beginner friendly)
- <a href ="https://www.kaggle.com/learn/intro-to-machine-learning">Kaggle Courses: Intro to Machine Learning</a> (beginner friendly)
- <a href ="https://www.kaggle.com/learn/intermediate-machine-learning">Kaggle Courses: Intermediate Machine Learning</a>
- <a href ="https://www.kaggle.com/learn/feature-engineering">Kaggle Courses: Feature Engineering</a>
- <a href ="https://www.kaggle.com/learn/machine-learning-explainability">Kaggle Courses: Machine Learning Explainability</a>

**Inspiring Kaggle Notebooks**
- <a href ="https://www.kaggle.com/code/fabiendaniel/customer-segmentation/notebook">Customer-segmentation</a> by Fabien Daniel
- <a href="https://www.kaggle.com/code/anmoltripathi/complete-e-commerce-analysis">Complete E-Commerce Analysis</a> by Anmol Tripathi
- <a href="https://www.kaggle.com/code/allunia/e-commerce-sales-forecast/notebook">E-Commerce Sales Forecast</a> by Laura Fink
- <a href ="https://www.kaggle.com/code/sercanyesiloz/crm-analytics/notebook">CRM Analytics</a> by Sercan Yeşilöz

**Others**
- Segal, Troy (August 23, 2021). Investopedia.com, <a href ="https://www.investopedia.com/terms/r/rfm-recency-frequency-monetary-value.asp">Recency, Frequency, Monetary Value (RFM)</a>